# RiskEngine End-to-End (30–180d history → 90d risk)
This notebook builds a risk prediction pipeline over your four CSVs.

**Inputs expected:**
- `Patient_info.csv`
- `Biochemical_parameters.csv`
- `Diagnostics.csv`
- `Glucose_measurements.csv` (large; we'll aggregate to daily metrics)

**Outputs:**
- `outputs/feature_dataset.parquet` – modeling dataset with rolling-window features
- `outputs/model_calibrated.joblib` – trained + calibrated model
- `outputs/metrics.json` – AUROC, AUPRC, Brier, confusion matrix
- `outputs/plots/` – ROC, PR, Calibration plots


In [5]:
# --- Setup ---
import os, json, math, gc, sys, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Prefer xgboost; fallback to HistGradientBoosting if not available
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    from sklearn.ensemble import HistGradientBoostingClassifier
    HAS_XGB = False

try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False

BASE = Path(".").resolve()
OUT = BASE / "outputs"
PLOTS = OUT / "plots"
for p in [OUT, PLOTS]:
    p.mkdir(parents=True, exist_ok=True)

print("Using XGBoost:", HAS_XGB, "| SHAP available:", HAS_SHAP)


Using XGBoost: True | SHAP available: True


In [6]:
# --- Config: set your file paths here ---
PATH_PATIENT_INFO = "Patient_info.csv"
PATH_LABS = "Biochemical_parameters.csv"
PATH_DIAG = "Diagnostics.csv"
PATH_CGM = "Glucose_measurements.csv"   # large file

# CGM cadence assumption (15-min ~ 96/day)
EXPECTED_READS_PER_DAY = 96

# Rolling windows (in days)
WINDOWS = [30, 90, 180]

# Label thresholds
FUTURE_90D_HYPER_MEAN_THRESHOLD = 0.35   # avg future 90d fraction > 180 mg/dL
A1C_RISE_THRESHOLD = 0.5                 # A1c increase >= 0.5 absolute within 90d


In [7]:
# --- Load the three small tables ---
pi = pd.read_csv(PATH_PATIENT_INFO)
bp = pd.read_csv(PATH_LABS)
dg = pd.read_csv(PATH_DIAG)

# Parse dates in patient_info
for c in ["Initial_measurement_date","Final_measurement_date",
          "Initial_biochemical_parameters_date","Final_biochemical_parameters_date"]:
    if c in pi.columns:
        pi[c] = pd.to_datetime(pi[c], dayfirst=True, errors="coerce")

# Parse labs
if "Reception_date" in bp.columns:
    bp["Reception_date"] = pd.to_datetime(bp["Reception_date"], dayfirst=True, errors="coerce")

print("Patients:", pi['Patient_ID'].nunique(), 
      "| Labs coverage:", bp['Patient_ID'].nunique(), 
      "| Diagnostics coverage:", dg['Patient_ID'].nunique())
bp.head(3), dg.head(3), pi.head(3)


Patients: 736 | Labs coverage: 723 | Diagnostics coverage: 511


(  Patient_ID Reception_date                              Name  Value
 0  LIB193265     2018-05-09                         Potassium    4.5
 1  LIB193265     2018-05-09                   HDL cholesterol   64.0
 2  LIB193265     2018-05-09  Gamma-glutamyl Transferase (GGT)   11.0,
   Patient_ID    Code                                        Description
 0  LIB193263   272.4               Other and unspecified hyperlipidemia
 1  LIB193264   354.0                             Carpal tunnel syndrome
 2  LIB193264  574.00  Calculus of gallbladder with acute cholecystit...,
   Patient_ID Sex  Birth_year Initial_measurement_date Final_measurement_date  \
 0  LIB193263   M        1965               2020-09-06             2022-03-19   
 1  LIB193264   F        1975               2020-10-06             2022-03-19   
 2  LIB193265   F        1980                      NaT             2022-03-19   
 
    Number_of_days_with_measures  Number_of_measurements  \
 0                           648        

In [ ]:
    # --- Aggregate CGM to daily features (chunked) ---
    # Produces a per-patient-per-day summary to keep memory in check.

    def aggregate_cgm_daily(path_cgm, expected_reads_per_day=96, chunksize=2_000_000):
        cols = ["Patient_ID","Measurement_date","Measurement_time","Measurement"]
        usecols = None  # if your file has extra cols, we can restrict: usecols=cols
        dtypes = {"Patient_ID":"category", "Measurement":"float32"}
        out_frames = []

        # We'll accumulate in a dict to avoid too many frames
        agg = {}

        for chunk in pd.read_csv(path_cgm, usecols=usecols, dtype=dtypes,
                                chunksize=chunksize, low_memory=True):
            # combine date + time into timestamp then extract date
            # note: parsing as datetime is expensive; we only need date-level
            # We'll keep Measurement_date (dayfirst) and ignore time for daily group
            chunk["Measurement_date"] = pd.to_datetime(chunk["Measurement_date"], dayfirst=True, errors="coerce").dt.date
            chunk = chunk.dropna(subset=["Measurement_date","Measurement","Patient_ID"])
            # basic flags
            chunk["gt_180"] = (chunk["Measurement"] > 180).astype("int8")
            chunk["lt_70"]  = (chunk["Measurement"] < 70).astype("int8")
            # groupby patient/day
            grp = chunk.groupby(["Patient_ID","Measurement_date"]).agg(
                n_reads=("Measurement","size"),
                mean_glu=("Measurement","mean"),
                std_glu=("Measurement","std"),
                max_glu=("Measurement","max"),
                min_glu=("Measurement","min"),
                gt_180_sum=("gt_180","sum"),
                lt_70_sum=("lt_70","sum"),
            ).reset_index()

            for _, r in grp.iterrows():
                key = (r["Patient_ID"], r["Measurement_date"])
                if key not in agg:
                    agg[key] = {
                        "Patient_ID": r["Patient_ID"],
                        "date": pd.to_datetime(str(r["Measurement_date"])),
                        "n_reads": int(r["n_reads"]),
                        "sum_glu": float(r["mean_glu"] * r["n_reads"]),
                        "sum_sq_glu": float((r["std_glu"]**2 + r["mean_glu"]**2) * r["n_reads"]) if not pd.isna(r["std_glu"]) else float(r["mean_glu"]**2 * r["n_reads"]),
                        "max_glu": float(r["max_glu"]),
                        "min_glu": float(r["min_glu"]),
                        "gt_180": int(r["gt_180_sum"]),
                        "lt_70": int(r["lt_70_sum"]),
                    }
                else:
                    a = agg[key]
                    a["n_reads"] += int(r["n_reads"])
                    a["sum_glu"] += float(r["mean_glu"] * r["n_reads"])
                    a["sum_sq_glu"] += float((r["std_glu"]**2 + r["mean_glu"]**2) * r["n_reads"]) if not pd.isna(r["std_glu"]) else float(r["mean_glu"]**2 * r["n_reads"])
                    a["max_glu"] = max(a["max_glu"], float(r["max_glu"]))
                    a["min_glu"] = min(a["min_glu"], float(r["min_glu"]))
                    a["gt_180"] += int(r["gt_180_sum"])
                    a["lt_70"] += int(r["lt_70_sum"])

            del chunk, grp
            gc.collect()

        # materialize
        rows = []
        for (pid, day), a in agg.items():
            mean = a["sum_glu"] / max(1, a["n_reads"])
            # variance = E[x^2] - (E[x])^2
            var = max(0.0, a["sum_sq_glu"]/max(1, a["n_reads"]) - mean**2)
            std = math.sqrt(var)
            rows.append({
                "Patient_ID": pid,
                "date": a["date"],
                "n_reads": a["n_reads"],
                "coverage": min(1.0, a["n_reads"] / expected_reads_per_day),
                "mean_glu": mean,
                "std_glu": std,
                "max_glu": a["max_glu"],
                "min_glu": a["min_glu"],
                "frac_gt_180": a["gt_180"] / max(1, a["n_reads"]),
                "frac_lt_70": a["lt_70"] / max(1, a["n_reads"]),
                "tir_70_180": max(0.0, 1.0 - (a["gt_180"] + a["lt_70"]) / max(1, a["n_reads"])),
            })
        daily = pd.DataFrame(rows).sort_values(["Patient_ID","date"])
        return daily

    # NOTE: Running this on the full CGM file can take time. Uncomment when ready.
    # daily_cgm = aggregate_cgm_daily(PATH_CGM, EXPECTED_READS_PER_DAY)
    # daily_cgm.to_parquet(OUT / "daily_cgm.parquet", index=False)
    print("If ready, run aggregate_cgm_daily(PATH_CGM) to produce daily aggregates → outputs/daily_cgm.parquet")


If ready, run aggregate_cgm_daily(PATH_CGM) to produce daily aggregates → outputs/daily_cgm.parquet


In [9]:
# --- Quick load of pre-aggregated daily CGM if you've already generated it ---
daily_path = OUT / "daily_cgm.parquet"
if daily_path.exists():
    daily_cgm = pd.read_parquet(daily_path)
    print("Loaded daily CGM:", daily_cgm.shape)
    display(daily_cgm.head())
else:
    print("Daily CGM parquet not found. Generate it in the previous cell when you have the big file locally.")


Daily CGM parquet not found. Generate it in the previous cell when you have the big file locally.


In [8]:
# --- Prepare A1c time series per patient (for features + label proxy) ---
# We filter labs to A1c and keep (Patient_ID, date, value)
a1c = bp[bp["Name"].str.contains("Glycated hemoglobin", case=False, na=False)].copy()
a1c = a1c.rename(columns={"Reception_date":"date","Value":"a1c"})
a1c = a1c[["Patient_ID","date","a1c"]].dropna()
a1c = a1c.sort_values(["Patient_ID","date"])
print("A1c rows:", a1c.shape)
display(a1c.head())


A1c rows: (1963, 3)


,Patient_ID,date,a1c
11,LIB193265,2018-05-09,8.6
33,LIB193265,2019-12-03,7.5
60,LIB193265,2020-02-06,7.3
165,LIB193266,2021-05-04,7.5
169,LIB193266,2021-05-11,7.5


In [9]:
# --- Diagnostics flags (static, per patient) ---
# Create binary flags for top-15 most frequent codes to avoid high dimensionality.
top_codes = dg["Code"].value_counts().head(15).index.tolist()
diag_flags = (dg.assign(val=1)
                .pivot_table(index="Patient_ID", columns="Code", values="val", aggfunc="max", fill_value=0))
# keep only top codes
diag_flags = diag_flags[[c for c in diag_flags.columns if c in top_codes]]
diag_flags = diag_flags.add_prefix("diag_").reset_index()
print("Diagnostics one-hot shape:", diag_flags.shape)
display(diag_flags.head())


Diagnostics one-hot shape: (511, 16)


Code,Patient_ID,diag_244.9,diag_250.4,diag_250.5,diag_250.51,diag_250.6,diag_272.0,diag_272.4,diag_278.00,diag_354.0,diag_362.01,diag_401.9,diag_477.0,diag_477.9,diag_493.9,diag_733.00
0,LIB193263,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,LIB193264,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,LIB193266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,LIB193267,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LIB193269,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# --- Build rolling-window features and labels ---
if 'daily_cgm' not in globals():
    raise RuntimeError("daily_cgm not loaded. Generate or load outputs/daily_cgm.parquet first.")

# Helper: compute backward rolling (30/90/180) for select columns per patient
def add_rolling_features(df, cols, windows):
    df = df.sort_values(["Patient_ID","date"]).copy()
    for w in windows:
        for c in cols:
            df[f"{c}_mean_{w}"] = (df.groupby("Patient_ID")[c]
                                     .transform(lambda s: s.rolling(window=w, min_periods=7).mean()))
            df[f"{c}_std_{w}"] = (df.groupby("Patient_ID")[c]
                                     .transform(lambda s: s.rolling(window=w, min_periods=7).std()))
        # slope on mean_glu over window w
        def slope_rolling(x):
            # compute slope using indices 0..len-1 inside window
            vals = pd.Series(x)
            out = np.full(len(vals), np.nan, dtype="float64")
            for i in range(len(vals)):
                start = max(0, i - w + 1)
                window_vals = vals[start:i+1]
                if window_vals.notna().sum() >= 7:
                    y = window_vals.dropna().values
                    t = np.arange(len(y)).reshape(-1,1)
                    lr = LinearRegression().fit(t, y)
                    out[i] = lr.coef_[0]
            return pd.Series(out, index=vals.index)
        df[f"mean_glu_slope_{w}"] = df.groupby("Patient_ID")["mean_glu"].transform(slope_rolling)
    return df

# Compute rolling features on daily CGM (backward-looking, ending at date d)
feat_cols = ["mean_glu","std_glu","tir_70_180","frac_gt_180","frac_lt_70","coverage","n_reads"]
daily_feat = add_rolling_features(daily_cgm, feat_cols, WINDOWS)

# Align features to an index date = day + 1
daily_feat["index_date"] = daily_feat["date"] + pd.Timedelta(days=1)

# Add patient-level demographics (age approx from Birth_year)
pi_dem = pi[["Patient_ID","Sex","Birth_year"]].copy()
daily_feat = daily_feat.merge(pi_dem, on="Patient_ID", how="left")

# Add A1c latest prior to index_date using merge_asof
a1c_sorted = a1c.sort_values(["Patient_ID","date"])
daily_feat = daily_feat.sort_values(["Patient_ID","index_date"])
daily_feat = pd.merge_asof(
    daily_feat, a1c_sorted, left_on="index_date", right_on="date",
    by="Patient_ID", direction="backward"
).rename(columns={"a1c":"a1c_prior"}).drop(columns=["date_y"], errors="ignore")

# Add diag flags (static)
daily_feat = daily_feat.merge(diag_flags, on="Patient_ID", how="left").fillna(0)

# --- Build future 90d label proxies ---
# 1) Hyperglycemia burden: average frac_gt_180 over next 90 days >= threshold
def future_mean_next_n_days(group, col, n=90):
    # compute forward rolling mean by reversing
    s = group[col].astype(float).copy()
    s_rev = s.iloc[::-1].rolling(window=n, min_periods=7).mean().iloc[::-1]
    # this mean at day d includes d and next (n-1) days; shift by -1 so it starts next day
    fut = s_rev.shift(-1)
    return fut

daily_feat = daily_feat.sort_values(["Patient_ID","date_x"]).rename(columns={"date_x":"date"})
daily_feat["future90_frac_gt_180_mean"] = (daily_feat
    .groupby("Patient_ID", group_keys=False)
    .apply(lambda g: future_mean_next_n_days(g, "frac_gt_180", 90))
)

# 2) A1c rise in the next 90 days: find next A1c within 90 days via forward merge_asof with tolerance
a1c_sorted = a1c_sorted.rename(columns={"date":"a1c_date","a1c":"a1c_value"})
# forward asof requires aligning right_on >= left_on; we'll merge on index_date and use direction='forward'
tmp = pd.merge_asof(
    daily_feat[["Patient_ID","index_date"]].sort_values(["Patient_ID","index_date"]),
    a1c_sorted.sort_values(["Patient_ID","a1c_date"]),
    left_on="index_date", right_on="a1c_date", by="Patient_ID", direction="forward",
    tolerance=pd.Timedelta(days=90)
)
daily_feat["a1c_future"] = tmp["a1c_value"].values

# Label: 1 if (future90 hyper mean >= threshold) OR (a1c_future - a1c_prior >= A1C_RISE_THRESHOLD)
daily_feat["label_90d"] = (
    (daily_feat["future90_frac_gt_180_mean"] >= FUTURE_90D_HYPER_MEAN_THRESHOLD) |
    ((daily_feat["a1c_future"] - daily_feat["a1c_prior"]) >= A1C_RISE_THRESHOLD)
).astype(int)

# Keep only rows with at least 30 days of history and 90 days of future available
# Approximation: require non-null rolling features for 30d window and non-null future stat
mask_hist = daily_feat["mean_glu_mean_30"].notna()
mask_future = daily_feat["future90_frac_gt_180_mean"].notna()
dataset = daily_feat[mask_hist & mask_future].copy()

feature_cols = [c for c in dataset.columns if any(x in c for x in ["_mean_","_std_","_slope_","coverage","n_reads"]) and "future90" not in c]
# Add demographics and diag flags
feature_cols += ["Birth_year"] + [c for c in dataset.columns if c.startswith("diag_")]
# encode Sex simply (M/F/other)
if "Sex" in dataset.columns:
    dataset["Sex_enc"] = dataset["Sex"].astype("category").cat.codes
    feature_cols += ["Sex_enc"]

keep_cols = ["Patient_ID","date","index_date","label_90d"] + feature_cols + ["a1c_prior","a1c_future"]
dataset = dataset[keep_cols].dropna(subset=feature_cols, how="all")

print("Modeling dataset shape:", dataset.shape)
dataset.head(3)


RuntimeError: daily_cgm not loaded. Generate or load outputs/daily_cgm.parquet first.

In [ ]:
# --- Train, calibrate, evaluate ---
# Patient-level split to avoid leakage
patients = dataset["Patient_ID"].unique()
rng = np.random.default_rng(42)
test_pat = set(rng.choice(patients, size=int(0.2*len(patients)), replace=False))

train_df = dataset[~dataset["Patient_ID"].isin(test_pat)].copy()
test_df  = dataset[ dataset["Patient_ID"].isin(test_pat)].copy()

y_tr = train_df["label_90d"].values
y_te = test_df["label_90d"].values

X_tr = train_df[feature_cols]
X_te = test_df[feature_cols]

# simple preprocessing (impute medians)
from sklearn.preprocessing import StandardScaler
pre = ColumnTransformer([("num", SimpleImputer(strategy="median"), feature_cols)], remainder="drop")

if HAS_XGB:
    clf = XGBClassifier(
        n_estimators=400, max_depth=4, learning_rate=0.06,
        subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
        tree_method="hist", random_state=7, eval_metric="logloss",
        scale_pos_weight=(y_tr==0).sum()/(y_tr==1).sum() if (y_tr==1).sum()>0 else 1.0
    )
else:
    from sklearn.ensemble import HistGradientBoostingClassifier
    clf = HistGradientBoostingClassifier(max_depth=3, learning_rate=0.06, max_iter=400, random_state=7)

pipe = Pipeline([("pre", pre), ("clf", clf)])
pipe.fit(X_tr, y_tr)

# Calibrate
cal = CalibratedClassifierCV(pipe, cv=3, method="isotonic")
cal.fit(X_tr, y_tr)

# Evaluate
proba = cal.predict_proba(X_te)[:,1]
auroc = roc_auc_score(y_te, proba) if len(np.unique(y_te))>1 else float("nan")
auprc = average_precision_score(y_te, proba) if len(np.unique(y_te))>1 else float("nan")
brier = brier_score_loss(y_te, proba)

thr = 0.5
pred = (proba >= thr).astype(int)
cm = confusion_matrix(y_te, pred).tolist()

print("AUROC:", auroc, "AUPRC:", auprc, "Brier:", brier, "CM@0.5:", cm)

# Save artifacts
import joblib
joblib.dump(cal, OUT / "model_calibrated.joblib")
with open(OUT / "metrics.json","w") as f:
    json.dump({"AUROC":auroc, "AUPRC":auprc, "Brier":brier, "ConfusionMatrix@0.5":cm}, f, indent=2)
with open(OUT / "feature_meta.json","w") as f:
    json.dump({"feature_cols": feature_cols}, f, indent=2)

# Save dataset for app/demo
dataset.to_parquet(OUT / "feature_dataset.parquet", index=False)

# Plots (matplotlib only)
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
fig, ax = plt.subplots()
RocCurveDisplay.from_predictions(y_te, proba, ax=ax); plt.savefig(PLOTS / "roc_curve.png"); plt.close()
fig, ax = plt.subplots()
PrecisionRecallDisplay.from_predictions(y_te, proba, ax=ax); plt.savefig(PLOTS / "pr_curve.png"); plt.close()

# Calibration plot
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y_te, proba, n_bins=10, strategy='quantile')
plt.plot(prob_pred, prob_true, marker='o'); plt.plot([0,1],[0,1],'--')
plt.xlabel("Predicted"); plt.ylabel("Observed"); plt.title("Calibration")
plt.savefig(PLOTS / "calibration.png"); plt.close()

print("Artifacts saved in:", OUT)


In [ ]:
# --- Explainability (SHAP) ---
if HAS_SHAP:
    # SHAP with predict_proba callable works through calibrated pipeline
    # For speed, sample
    sample = train_df.sample(n=min(200, len(train_df)), random_state=7)
    Xs = sample[feature_cols]
    explainer = shap.Explainer(cal.predict_proba, Xs, feature_names=feature_cols)
    sv = explainer(Xs)
    shap.plots.beeswarm(sv, max_display=20, show=False)
    plt.tight_layout(); plt.savefig(PLOTS / "shap_global.png"); plt.close()
    print("Saved SHAP beeswarm to", PLOTS / "shap_global.png")
else:
    print("SHAP not installed; skip explainability here. (Install shap to enable.)")
